In [5]:
from pydub import AudioSegment

In [11]:
for i in range(1, 61):
    music = AudioSegment.from_file("/home/rajesh/MusicSpeechSeparartion/Dataset/music/"+str(i)+".wav", format="wav")
    speech = AudioSegment.from_file("/home/rajesh/MusicSpeechSeparartion/Dataset/speech/"+str(i)+".wav", format="wav")

    merge = music+speech
    mixture = merge.export("/home/rajesh/MusicSpeechSeparartion/Dataset/Mixture/"+str(i)+".wav", format="wav")
    
    speech = speech + 6
    overlay = speech.overlay(music, position=0)
    output = overlay.export("/home/rajesh/MusicSpeechSeparartion/Dataset/Overlay/"+str(i)+".wav", format="wav")
